In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 20000
batch_size = 128
display_step = 10

# Network parameters
n_inputs = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.85 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [ ]:
dropout = 0.65 # Dropout, probability to keep units

In [4]:
# Create some wrapper for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool wrapper
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [5]:
# Convnet: conv1->maxpool1->conv2->maxpool2->fc->dropout->output
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # Conv1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # maxpool1
    conv1 = maxpool2d(conv1, k=2)
    
    # Conv2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # maxpool2
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer
    # reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# Store layers weight $ biases
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5,5,32, 64])),
    # fully connceted, 7x7x64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [7]:
# construct model
pred = conv_net(x, weights, biases, keep_prob)

# define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [9]:
# Lauch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (back prop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        if step % display_step == 0:
            # calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            print("Iter ", str(step*batch_size), ", Minibatch Loss= ", "{:.6f}".format(loss), 
                  ", Training Accuracy: ", "{:.5f}".format(acc))
        step += 1
    print("Optimization Fininshed!")
        
    # Calculate accuracy for 256 mnist test images
    print("Test Accuracy: ", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: dropout}))

Iter  1280 , Minibatch Loss=  26807.210938 , Training Accuracy:  0.28906
Iter  2560 , Minibatch Loss=  18369.023438 , Training Accuracy:  0.42188
Iter  3840 , Minibatch Loss=  13674.798828 , Training Accuracy:  0.48438
Iter  5120 , Minibatch Loss=  12245.734375 , Training Accuracy:  0.60156
Iter  6400 , Minibatch Loss=  11329.076172 , Training Accuracy:  0.58594
Iter  7680 , Minibatch Loss=  4430.258301 , Training Accuracy:  0.78125
Iter  8960 , Minibatch Loss=  4808.968262 , Training Accuracy:  0.74219
Iter  10240 , Minibatch Loss=  5946.623535 , Training Accuracy:  0.69531
Iter  11520 , Minibatch Loss=  4242.037598 , Training Accuracy:  0.75781
Iter  12800 , Minibatch Loss=  3978.001465 , Training Accuracy:  0.76562
Iter  14080 , Minibatch Loss=  2873.983887 , Training Accuracy:  0.84375
Iter  15360 , Minibatch Loss=  2679.179443 , Training Accuracy:  0.85156
Iter  16640 , Minibatch Loss=  3157.176270 , Training Accuracy:  0.80469
Iter  17920 , Minibatch Loss=  4857.800781 , Training